## **Q1**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# ID of the files on Google Drive
sales_data_id = '15uTTtFT-AkNYiH_DrmKoIBSq8PpO-e0Z'
stores_data_id = '1zIp7B3Lm1F8HE5mhcfAh30kqQMx7ttQs'

# Download 'sales-data-set.csv'
sales_downloaded = drive.CreateFile({'id': sales_data_id})
sales_downloaded.GetContentFile('sales-data-set.csv')

# Download 'stores-data-set.csv'
stores_downloaded = drive.CreateFile({'id': stores_data_id})
stores_downloaded.GetContentFile('stores-data-set.csv')

In [ ]:
# Initialize a Spark session
spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()

In [ ]:
# Load the sales data from 'sales-data-set.csv' into a DataFrame
sales_data = spark.read.csv("sales-data-set.csv", header=True, inferSchema=True)

# Load the store data from 'stores-data-set.csv' into a DataFrame
store_data = spark.read.csv("stores-data-set.csv", header=True, inferSchema=True)

**(a) The total sales for each store type. [10 pts (5 pts for code)] Note that the result should have 3 rows (header is not included)**

In [ ]:
# Join the two DataFrames on the 'Store' column to associate sales with store types
joined_data = sales_data.join(store_data, on="Store")

# Calculate the total sales for each store type and select only the relevant columns
total_sales_by_type = joined_data.groupBy("Type").agg(sum("Weekly_Sales").alias("Total_Sales"))

# Reorder the rows to display "A" first, "B" second, and "C" third
total_sales_by_type = total_sales_by_type.orderBy(col("Type"))

# Format the 'Total_Sales' column to display numbers with commas and two decimal places
total_sales_by_type = total_sales_by_type.withColumn("Total_Sales", format_number("Total_Sales", 2))

# Show the result (excluding the header)
total_sales_by_type.show(truncate=False)

+----+----------------+
|Type|Total_Sales     |
+----+----------------+
|A   |4,331,014,722.75|
|B   |2,000,700,736.82|
|C   |405,503,527.54  |
+----+----------------+



**(b) Average sales on Holidays vs. Non-Holidays, and determine if sales are generally higher during holidays. [10 pts (5 pts for code)] Note that the result should have 2 rows (header is not included)**

In [ ]:
# Initialize another Spark session
spark2 = SparkSession.builder.appName("HolidaySalesAnalysis").getOrCreate()

# Calculate average sales for holidays and non-holidays
result = sales_data.groupBy("IsHoliday")\
                  .agg(avg("Weekly_Sales").alias("Avg_Sales"))\
                  .withColumn("IsHoliday", when(col("IsHoliday") == "false", "Non-Holiday")
                                        .otherwise("Holiday"))

# Show the result
result.show()

+-----------+------------------+
|  IsHoliday|         Avg_Sales|
+-----------+------------------+
|    Holiday| 17035.82318735042|
|Non-Holiday|15901.445069008514|
+-----------+------------------+

